In [ ]:
import sys
from pathlib import Path

# Add root folder to Python path (to import modules)
notebook_dir = Path().absolute()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pandas as pd
import torchmetrics as tm
import numpy as np
import torch
import models.neural_net.metric as metric
from src.portfolios.portfolio_new import Portfolio

In [ ]:
predicted_ret_path = '../saved/final_results/predicted_returns/'
# predicted_ret_path = '../saved/results/FinalExpandingTuning/predicted_returns/'
predicted_ret_files = [f for f in listdir(predicted_ret_path) if isfile(join(predicted_ret_path, f))]

In [ ]:
results_df = pd.read_csv('../saved/final_results/results_df.csv', index_col=0)
# results_df.loc[results_df.trial_id == 16669034630977150]

In [ ]:
pd.set_option('display.max_columns', None)
results_df.dropna()
#pd.reset_option('max_columns')


In [ ]:
# Find best network
results_df = results_df.dropna()
best_net = results_df.loc[results_df['oosSpearman'] == results_df['oosSpearman'].max()]
worst_net = results_df.loc[results_df['oosSpearman'] == results_df['oosSpearman'].min()]
average_net = results_df.drop('trial_id', axis=1).mean(axis=0)

In [ ]:
best_net

In [ ]:
worst_net.dropna()

# Plot portfolio returns

In [ ]:
from portfolios.market_portfolio import MarketPortfolio
base_path = '/home/ge65cuw/thesis/saved/final_results/'
best_net_path = base_path + 'ensembleNN3.csv'
worst_net_path = base_path + 'predicted_returns/' + str(worst_net.trial_id.values[0]) + '_predicted_returns.csv'
benchmark_nn4_path = base_path + 'ensembleGu.csv'
linear_path = base_path + 'linear/predicted_ret.csv'
mkt = MarketPortfolio()

In [ ]:
best = pd.read_csv(best_net_path, index_col=0)
worst = pd.read_csv(worst_net_path, index_col=0)
benchmark = pd.read_csv(benchmark_nn4_path, index_col=0)

In [ ]:
pbest = Portfolio(best, verbose=0)
pworst = Portfolio(worst, verbose=0)
pbenchmark = Portfolio(benchmark, verbose=0)

In [ ]:
rbest = pbest.returns[['yyyymm','l-s']].rename({'l-s': 'best'}, axis=1)
rworst = pworst.returns[['yyyymm','l-s']].rename({'l-s': 'worst'}, axis=1)
rbenchmark = pbenchmark.returns[['yyyymm','l-s']].rename({'l-s': 'benchmark'}, axis=1)

In [ ]:
rbest

In [ ]:
mkt_ret = mkt.mkt_returns

portfolio = rbest.merge(rworst, on=['yyyymm'], how='left')
portfolio = portfolio.merge(rbenchmark, on='yyyymm', how='left')
portfolio = portfolio.merge(mkt_ret, on='yyyymm', how='left')
cum_returns = portfolio.copy()
cum_returns = cum_returns.loc[cum_returns.yyyymm >= 200001].copy()
cum_returns.iloc[:,1:] = np.log(cum_returns.iloc[:,1:]/100+1).cumsum()

In [ ]:
cum_returns

In [ ]:
path = '/home/ge65cuw/thesis/saved/final_results/img/cum_ret_plot.png'

In [ ]:
from cycler import cycler
import datetime as dt
import matplotlib.pyplot as plt


# cum_returns = cum_returns.merge(mkt_ret, on=['yyyymm'], how='left')
# cum_returns.iloc[:,-1] = (np.log(cum_returns.iloc[:,-1]/100+1).cumsum())

cum_returns['date'] = cum_returns['yyyymm'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m'))
# print('Self.cum_returns columns are:')
# print(self.cum_returns.columns)
linestyle_cycler = (cycler('color', ['deepskyblue','coral','magenta','royalblue', 'red','lime', 'crimson', 'cyan','springgreen','teal','gray','darkorange']) +
                    cycler('linestyle',['-','--',':','-.',':','-','-.','--','-',':','-.','--']))


fig = plt.figure(figsize=(15, 5))
ax = plt.gca()
ax.set_prop_cycle(linestyle_cycler)
plt.plot(cum_returns['date'], cum_returns.iloc[:,1:-1]) # plt.plot(l, ret.iloc[:,1:])
plt.ylabel('Cumulative Log returns', fontsize=14)
plt.legend(['Best Network', 'Worst Network', 'Benchmark NN4', 'Market Return'], fontsize=12)
plt.tight_layout()
plt.savefig(path)
plt.show()
# try:
#     plt.savefig(path)
# except:
#     pass


In [ ]:
returns = best.copy()
returns = returns.rename({'predicted_ret':'best'}, axis=1)
returns = returns.merge(worst.drop('ret',axis=1), on=['permno','yyyymm'], how='left').rename({'predicted_ret':'worst'},axis=1)
returns = returns.merge(benchmark.drop('ret',axis=1), on=['permno','yyyymm'], how='left').rename({'predicted_ret':'benchmark'},axis=1)

In [ ]:
import data.data_preprocessing as dp
crsp = dp.load_crsp()

In [ ]:
returns

In [ ]:
returns = returns.drop('ret',axis=1).merge(crsp[['permno','yyyymm','ret']], on=['permno','yyyymm'], how='left')

In [ ]:
# Remove the plot frame lines. They are unnecessary chartjunk.
# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]  

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.  
for i in range(len(tableau20)):  
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

# You typically want your plot to be ~1.33x wider than tall. This plot is a rare  
# exception because of the number of lines being plotted on it.  
# Common sizes: (10, 7.5) and (12, 9)  
plt.figure(figsize=(8, 5.5))

# Remove the plot frame lines. They are unnecessary chartjunk.  
ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["bottom"].set_visible(True)  
ax.spines["right"].set_visible(False)  
ax.spines["left"].set_visible(True)  

ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()



# Distribution of actual returns and predicted returns
bins = np.linspace(-20, 20, 5000)
# bins2 = np.linspace(-20,20,10000)
# bins = 1000
x1 = returns['best']
x2 = returns['worst']
x3 = returns['benchmark']
x4 = returns['ret']
plt.hist(x2, bins, label='WNN', color='green')
plt.hist(x3, bins, label='BNN', color='blue')#, edgecolor='blue')
plt.hist(x1, bins, alpha=0.8, label='TNN', color=tableau20[3])
plt.hist(x4, bins, alpha=0.5, label='Actual', color=tableau20[18])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# plt.xticks(range(-30, 30, 5), fontsize=13)
# plt.yticks(range(0, 120000, 20000), fontsize=13)
plt.legend(loc='upper right', fontsize=14)
plt.xlabel("Monthly stock returns (%)", fontsize=15)
# plt.ylim(1, 1750)
# plt.xlim(-10, 10)

plt.tight_layout()
plt.savefig('/home/ge65cuw/thesis/output.png', dpi= 400)
plt.show()
# bbox_inches="tight" removes all the extra whitespace on the edges of your plot.
# plt.savefig("Returns_distribution.png", bbox_inches="tight")
